<a href="https://colab.research.google.com/github/anvargasch/IEStoPRBT/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clone the repository containing the functions and photometric sample files.

In [1]:
!git clone https://github.com/anvargasch/IEStoPRBT.git

Cloning into 'IEStoPRBT'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), 139.73 KiB | 1.06 MiB/s, done.


In [2]:
from IEStoPRBT import readIES

# Example of reading an IES file and downloading the header and intensity matrix

In [ ]:
filename = '/content/IEStoPRBT/IESfiles/IESsiemens.ies'  # Replace with the path to your IES file
Lumen,fact,Num_vert_angle,Num_hori_angle,unid = readIES.IES_header(filename)
print("Lumen:",Lumen,", Factor:", fact,", #Vert:",Num_vert_angle ,", #Horiz:",Num_hori_angle,", Unidades:", unid, "(1:pies, 2:Metros)")

In [ ]:
filename = '/content/IEStoPRBT/IESfiles/IESsiemens.ies'
img=readIES.IES2PBRT (filename)

In [ ]:
import matplotlib.pyplot as plt   # Graphing functions
plt.figure(1)
plt.imshow(img ,cmap=plt.cm.gray), plt.title("Imagen IES")
plt.imshow

In [ ]:
readIES.save_EXR(img,'/content/IMG_EXR10.exr')